In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import cv2
import pandas as pd
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)

In [ ]:
CURRENT = r"/content/gdrive/MyDrive/Eye"
# PATH = r"/content/gdrive/MyDrive/Neural Nets/Eye/train_images_resized_preprocessed"

In [ ]:
def load_data():
    os.chdir(CURRENT)
    file = open('df_train_train', 'rb')
    df_train_train = pickle.load(file)
    file.close()

    file = open('df_train_valid', 'rb')
    df_train_test = pickle.load(file)
    file.close()
    
    return df_train_train,df_train_test

In [ ]:
df_train_train, df_train_test = load_data()

In [ ]:
df_train_train

In [ ]:
plt.hist(df_train_train['diagnosis'])
plt.show()

In [ ]:
BATCH_SIZE = 50
HEIGHT = 224
WIDTH = 224

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,validation_split = 0.0) #rotation_range=30,fill_mode='nearest',horizontal_flip=True
train_generator=train_datagen.flow_from_dataframe(dataframe=df_train_train,
                                                      directory="./train_images_resized_preprocessed/",
                                                      x_col="file_name",
                                                      y_col="diagnosis",
                                                      batch_size=BATCH_SIZE,
                                                      class_mode="categorical",
                                                      target_size=(HEIGHT, WIDTH),
                                                      subset='training')
valid_datagen=ImageDataGenerator(rescale=1./255,validation_split=0.99999)
valid_generator=valid_datagen.flow_from_dataframe(dataframe=df_train_test,
                                                      directory="./valid_images_resized_preprocessed/",
                                                      x_col="file_name",
                                                      y_col="diagnosis",
                                                      batch_size=BATCH_SIZE,
                                                      class_mode="categorical",    
                                                      target_size=(HEIGHT, WIDTH),
                                                      subset='validation')

In [ ]:
model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling=None,
    classes=1000,
)

In [ ]:
def CNN(model):
    x = model.outputs[-1]
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(5, activation="softmax")(x)

    model = models.Model(inputs=model.inputs, outputs=x)
    return model

In [ ]:
model = CNN(model)
model.summary()

In [ ]:
HIST = []
for c, (epoch, lr) in enumerate(zip([1, 5, 3], [1e-3, 1e-4, 1e-5])):  
  model.compile(optimizer=Adam(learning_rate=lr), loss=CategoricalCrossentropy(), metrics="accuracy")
  history = model.fit(train_generator, epochs=epoch, validation_data = valid_generator, verbose=1)
  HIST.append(history)
  np.save("./InceptionNet_history"+str(c)+".npy", np.array(HIST))
  model.save("./Inception"+str(c)+".h5")

59/59 [==============================] - 83s 715ms/step - loss: 0.6998 - accuracy: 0.7463 - val_loss: 81.6487 - val_accuracy: 0.5301
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_10
.........vars
......activation_11
.........vars
......activation_12
.........vars
......activation_13
.........vars
......activation_14
.........vars
......activation_15
.........vars
......activation_16
.........vars
......activation_17
.........vars
......activation_18
.........vars
......activation_19
.........vars
......activation_2
.........vars
......activation_20
.........vars
......activation_21
.........vars
......activation_22
.........vars
......activation_23
.........vars
......activation_24
.........vars
......activation_25
.........vars
......activation_26
.........vars
......activation_27
.........vars
......activation_28
.........vars
......activation_29
.........vars
......activati

KeyboardInterrupt: ignored

In [ ]:
np.save("./InceptionNet_history.npy", np.array(HIST))
model.save("./Inception.h5")